In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Setup for Google Colab directories
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
%cd MLexercise_CarDimensions

# import glob
# import re
import pandas as pd
dataframes = pd.read_csv("data/cleaned_data.csv")
print(dataframes.columns.tolist())
dataframes

/content/drive/MyDrive/Colab Notebooks/MLexercise_CarDimensions
['MAKE', 'MODEL', 'Model Year', 'Overall Length', 'Overall Width', 'Overall Height', 'Wheelbase', 'Curb Weight', 'Front End Length', 'Rear End Length', 'Side Glass Height', 'Body Side Height', 'Roof Width', 'Front Overhang', 'Rear Overhang', 'Track Width Front', 'Track Width Rear', 'Weight Distribution', 'Front Weight %', 'Rear Weight %']


,MAKE,MODEL,Model Year,Overall Length,Overall Width,Overall Height,Wheelbase,Curb Weight,Front End Length,Rear End Length,Side Glass Height,Body Side Height,Roof Width,Front Overhang,Rear Overhang,Track Width Front,Track Width Rear,Weight Distribution,Front Weight %,Rear Weight %
0,FIAT,124SPIDER2DRCONVERTIBLE,16,407.0,173.0,124.0,231.0,1114.0,155.0,79.0,31.0,67.0,116.0,85.0,91.0,152.0,150.0,54/46,54.0,46.0
1,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,208.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
2,RAMTRUCKS,1500CREWCAB57FTBOX2WD,18,592.0,202.0,197.0,367.0,2255.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
3,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,208.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
4,RAMTRUCKS,1500CREWCAB57FTBOX4X4,18,592.0,202.0,197.0,367.0,2327.0,130.0,321.0,45.0,97.0,140.0,101.0,124.0,174.0,173.0,57/43,57.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3759,BMW,Z4SDRIVE35I35IS2DRCONVERTIBLERWD,9,424.0,179.0,129.0,250.0,1610.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3760,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1585.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3761,BMW,Z4SDRIVE35I35IS2DRCONVRWD,9,424.0,179.0,129.0,250.0,1590.0,156.0,73.0,24.0,74.0,125.0,86.0,89.0,151.0,156.0,50/50,50.0,50.0
3762,ACURA,ZDX4DRSUVAWDTECHELITE,10,488.0,199.0,160.0,275.0,2064.0,133.0,109.0,33.0,88.0,120.0,109.0,104.0,172.0,171.0,58/42,58.0,42.0


In [ ]:
# # Features and target variable
features = [
'Overall Length', 'Overall Width', 'Overall Height',
'Front End Length', 'Rear End Length',
'Side Glass Height', 'Body Side Height',
'Wheelbase', 'Front Overhang', 'Rear Overhang', # Sum = Overall Length
'Roof Width', 'Track Width Front', 'Track Width Rear', # ~ Overall Width
]

target = 'Curb Weight'

X = dataframes[features]
y = dataframes[target]


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.5, random_state=0)

In [ ]:
# Model layers and training
# from tensorflow.keras.regularizers import l2

model = Sequential([
    Dense(256, activation='relu',
          input_shape=(X_train.shape[1],)
          ),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),

    # Dense(512, activation='relu',
    #       input_shape=(X_train.shape[1],),
    #       kernel_regularizer=l2(0.001)
    #       ),
    # Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    # Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    # Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1)  # output layer for regression
])

# from tensorflow.keras.optimizers import Adam
# optimizer = Adam(learning_rate=0.01)
# model.compile(optimizer=optimizer, loss='mse')
model.compile(optimizer='adam', loss='mse') # Default learning_rate=0.001
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss', patience=15, restore_best_weights=True
)
epochs = 500
model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=32,
    validation_split=0.2,  # 20% of training data used for validation
    callbacks=[early_stop],
    verbose=1)

Epoch 1/500


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4101 - val_loss: 0.1824
Epoch 2/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1711 - val_loss: 0.1597
Epoch 3/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1708 - val_loss: 0.1427
Epoch 4/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1929 - val_loss: 0.1375
Epoch 5/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1161 - val_loss: 0.1395
Epoch 6/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1595 - val_loss: 0.1408
Epoch 7/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1392 - val_loss: 0.1090
Epoch 8/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0963 - val_loss: 0.1633
Epoch 9/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0957 - val_loss: 0.1127
Epoch 10/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0868 - val_loss: 0.1200
Epoch 11/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1099 - val_loss: 0.1205
Epoch 12/500
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0941 - val_lo

In [ ]:
y_pred = model.predict(X_scaled)

# Scales prediction values back to their original target
y_pred_ori = scaler_y.inverse_transform(y_pred)

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

print("R² Score:", r2_score(y, y_pred_ori))
print("MSE:", mean_squared_error(y, y_pred_ori))
print("MSE Scaled:", mean_squared_error(y_scaled, y_pred))
print("Mean target weight:", y.mean())
print("Mean target predicted:", y_pred_ori.mean())

# Need to use original value to count MAPE because scaling with reshape(-1, 1)involve values close to 0 and makes it blow up
mape = mean_absolute_percentage_error(y, y_pred_ori)
print("MAPE:", mape * 100, "%")
# for i in range(100):
#   print(y_test[i], y_pred[i])

R² Score: 0.9041588389641682
MSE: 15353.17300425688
MSE Scaled: 0.095841158489372
Mean target weight: 1808.4378320935175
Mean target predicted: 1789.0719
MAPE: 4.966033584172325 %


In [ ]:
from helpers.utils import compare_predictions

comparison = compare_predictions(y, y_pred_ori)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  print(comparison.head(100))
comparison.to_csv("results/weight_prediction_TF.csv", index=False)

    Actual    Predicted       Error    Error %
0   1114.0  1214.462158  100.462158   9.018147
1   2255.0  2405.394043  150.394043   6.669359
2   2255.0  2424.279541  169.279541   7.506853
3   2327.0  2405.394043   78.394043   3.368889
4   2327.0  2424.279541   97.279541   4.180470
5   2277.0  2484.987305  207.987305   9.134269
6   2277.0  2515.506592  238.506592  10.474598
7   2373.0  2484.987305  111.987305   4.719229
8   2373.0  2515.506592  142.506592   6.005335
9   2231.0  2476.254883  245.254883  10.993047
10  2198.0  2476.254883  278.254883  12.659458
11  2367.0  2476.254883  109.254883   4.615753
12  2494.0  2476.254883  -17.745117   0.711512
13  2358.0  2476.254883  118.254883   5.015050
14  2367.0  2476.254883  109.254883   4.615753
15  2177.0  2378.615234  201.615234   9.261150
16  2177.0  2396.973877  219.973877  10.104450
17  2308.0  2378.615234   70.615234   3.059586
18  2308.0  2396.973877   88.973877   3.855021
19  2231.0  2476.254883  245.254883  10.993047
20  2198.0  2